# Dependencies and Setup

In [1]:
import pandas as pd
import numpy as np

# Read School and Student Data File and store into Pandas Data Frames
schools_df = pd.read_csv("Resources/schools_complete.csv")
students_df = pd.read_csv("Resources/students_complete.csv")

# Combine the data into a single dataset
merged = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])

# Something to make my life easier later on when alphabetizing things
merged = merged.replace('9th', '09th')

In [2]:
merged.head(8)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,09th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,09th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,09th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635


# First part will be creating a table with district overview data

In [3]:
# Prepping/organizing data for district data table

schools = merged['school_name'].unique() 
num_schools = len(schools) 

students = merged['student_name']
num_students = len(students)

# budget calculations
budget_calc_df = merged.filter(items=['school_name', 'budget']).drop_duplicates(subset='school_name')
budget = budget_calc_df['budget'].sum()

# average scores
avg_math_score = merged['math_score'].mean()
avg_read_score = merged['reading_score'].mean()
avg_overall_score = (avg_math_score + avg_read_score) / 2

# percent pass each class
m = 0
m_scores = merged['math_score']
for i in m_scores: 
    if m_scores[i] >= 70:
        m = m + 1
per_pass_math = m / num_students     
    
r = 0
r_scores = merged['reading_score']
for j in r_scores:
    if r_scores[j] >= 70:
        r = r + 1
per_pass_read = r / num_students


In [6]:
# Creating table containing summary of key district metrics

District_Summary = {
    'Total Number of Schools' : num_schools,
    'Total Number of Students' : num_students,
    'Total Budget' : budget,
    'Average Math Score' : avg_math_score,
    'Average Reading Score' : avg_read_score,
    'Overall Average Score' : avg_overall_score,
    'Percentage of Students Passing Math' : per_pass_math,
    'Percentage of Students Passing Reading' : per_pass_read
}

summary_df = pd.DataFrame.from_dict(District_Summary, orient='index', columns=['Entire District'])
summary_df

,Entire District
Total Number of Schools,1.500000e+01
Total Number of Students,3.917000e+04
Total Budget,2.464943e+07
Average Math Score,7.898537e+01
Average Reading Score,8.187784e+01
Overall Average Score,8.043161e+01
Percentage of Students Passing Math,6.041869e-01
Percentage of Students Passing Reading,9.215726e-01


# Next part is to create a table with key metrics for each school

In [7]:
# Determing key metrics for each school
# Groupby element creation, start of summary table 

schools_grouped = merged.groupby("school_name")
school_metrics = pd.DataFrame(schools_grouped['student_name'].count())

In [8]:
# Calculations for percent passing each class

t = 0
npm = 0
npr = 0
percent_passing_reading = []
percent_passing_math = []
m_grades = [] 
r_grades = []
s_type = []

for school, data in schools_grouped:
    m_grades = data['math_score']
    r_grades = data['reading_score']
    for i in m_grades:
        if i >= 70:
            npm = npm + 1
        t = t + 1
    for j in r_grades:
        if j >= 70:
            npr = npr + 1
    percent_passing_math.append(npm/t)
    percent_passing_reading.append(npr/t)
    m_grades = []
    r_grades = []

In [9]:
# Determine school type for each school

s = []
school_type = []

for school, data in schools_grouped:
    p = data['type'].unique()
    s.append(p)

for k in s:
    for u in k:
        school_type.append(u)

In [10]:
# Adding all the new columns

school_metrics['School Type'] = school_type
school_metrics['Total School Budget'] = schools_grouped['budget'].sum()
school_metrics['Budget per Student'] = school_metrics['Total School Budget'] / school_metrics['student_name']
school_metrics['Average Math Score'] = schools_grouped['math_score'].mean()
school_metrics['Average Reading Score'] = schools_grouped['reading_score'].mean()
school_metrics['Percent Passing Math'] = percent_passing_math
school_metrics['Percent Passing Reading'] = percent_passing_reading
school_metrics['Overall Passing Rate'] = (school_metrics['Percent Passing Math'] + school_metrics['Percent Passing Reading']) / 2

school_metrics = school_metrics.rename(columns={'student_name':'Number of Students'})
school_metrics

,Number of Students,School Type,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,4976,District,15549641728,3124928.0,77.048432,81.033963,0.666801,0.819333,0.743067
Cabrera High School,1858,Charter,2009159448,1081356.0,83.061895,83.975780,0.741440,0.860404,0.800922
Figueroa High School,2949,District,5557128039,1884411.0,76.711767,81.158020,0.716856,0.844424,0.780640
Ford High School,2739,District,4831365924,1763916.0,77.102592,80.746258,0.709471,0.833174,0.771322
Griffin High School,1468,Charter,1346890000,917500.0,83.351499,83.816757,0.733024,0.847677,0.790350
Hernandez High School,4635,District,14007062700,3022020.0,77.289752,80.934412,0.716725,0.837960,0.777342
Holden High School,427,Charter,105933149,248087.0,83.803279,83.814988,0.721394,0.840752,0.781073
Huang High School,2917,District,5573322295,1910635.0,76.629414,81.182722,0.712823,0.837089,0.774956
Johnson High School,4761,District,14733628650,3094650.0,77.072464,80.966394,0.703517,0.832660,0.768088


### Tables displaying the top 5 and bottom 5 performing schools, by overall passing rate

In [11]:
# Top 5 schools by overall passing rate

top_performers = school_metrics.nlargest(5, 'Overall Passing Rate')
top_performers

,Number of Students,School Type,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Wright High School,1800,Charter,1888920000,1049400.0,83.682222,83.955000,0.749809,0.858055,0.803932
Cabrera High School,1858,Charter,2009159448,1081356.0,83.061895,83.975780,0.741440,0.860404,0.800922
Wilson High School,2283,Charter,3012587442,1319574.0,83.274201,83.989488,0.740969,0.852850,0.796909
Griffin High School,1468,Charter,1346890000,917500.0,83.351499,83.816757,0.733024,0.847677,0.790350
Thomas High School,1635,Charter,1705517550,1043130.0,83.418349,83.848930,0.728104,0.845527,0.786816


In [12]:
# Bottom 5 schools by overall passing rate

bottom_performers = school_metrics.nsmallest(5, 'Overall Passing Rate')
bottom_performers

,Number of Students,School Type,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,4976,District,15549641728,3124928.0,77.048432,81.033963,0.666801,0.819333,0.743067
Johnson High School,4761,District,14733628650,3094650.0,77.072464,80.966394,0.703517,0.832660,0.768088
Rodriguez High School,3999,District,10186904637,2547363.0,76.842711,80.744686,0.705847,0.832665,0.769256
Ford High School,2739,District,4831365924,1763916.0,77.102592,80.746258,0.709471,0.833174,0.771322
Pena High School,962,Charter,563595396,585858.0,83.839917,84.044699,0.711938,0.837065,0.774502


## This will be a breakdown of reading and math scores by grade, for each school

In [13]:
by_grade = merged.groupby(['school_name','grade']).mean()
scores_by_grade = pd.DataFrame(by_grade)
scores_by_grade = scores_by_grade.drop(['size','budget','School ID','Student ID'], axis=1)
scores_by_grade.head(10)

reading_score  math_score
school_name          grade                           
Bailey High School   09th       81.303155   77.083676
                     10th       80.907183   76.996772
                     11th       80.945643   77.515588
                     12th       80.912451   76.492218
Cabrera High School  09th       83.676136   83.094697
                     10th       84.253219   83.154506
                     11th       83.788382   82.765560
                     12th       84.287958   83.277487
Figueroa High School 09th       81.198598   76.403037
                     10th       81.408912   76.539974

### Finally, a few tables to compare school performances based on budget, size, and school type

In [14]:
# Creating a table to break down school performance based on budget per student

by_budget = school_metrics
bins_budget = [0, 1000000, 2000000, 3000000, 50000000]
group_labels_budget = ['Under $1 million', "$1 to 2 million", '$2 to 3 million', 'Over $3 million']
by_budget['Budget Group'] = pd.cut(school_metrics['Budget per Student'], bins_budget, labels=group_labels_budget)
budget_group = by_budget.groupby('Budget Group').mean()
budget_df = pd.DataFrame(budget_group)
budget_df = budget_df.drop(['Number of Students', 'Total School Budget', 'Budget per Student'], axis=1)
budget_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Budget Group,,,,,
Under $1 million,83.664898,83.892148,0.722119,0.841831,0.781975
$1 to 2 million,80.904987,82.822740,0.727197,0.846352,0.786774
$2 to 3 million,76.842711,80.744686,0.705847,0.832665,0.769256
Over $3 million,77.136883,80.978256,0.695681,0.829984,0.762832


In [15]:
# Creating a table to break down school performance based on number of students

by_size = school_metrics
bins_size = [0, 1800, 3400, 5000]
group_labels_size = ['Small', 'Medium', 'Large']
by_size['Size Group'] = pd.cut(school_metrics['Number of Students'], bins_size, labels=group_labels_size)
size_groupby = by_size.groupby('Size Group').mean()
size_df = pd.DataFrame(size_groupby)
size_df = size_df.drop(['Number of Students', 'Total School Budget', 'Budget per Student'], axis=1)
size_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Size Group,,,,,
Small,83.575787,83.867683,0.727062,0.844728,0.785895
Medium,79.355974,82.210453,0.724312,0.845588,0.784950
Large,77.063340,80.919864,0.698222,0.830654,0.764438


In [16]:
# Creating a table to break down school performance based on school type (charter or district)

by_type = school_metrics
type_groupby = by_type.groupby('School Type').mean()
type_df = pd.DataFrame(type_groupby)
type_df = type_df.drop(['Number of Students', 'Total School Budget', 'Budget per Student'], axis=1)
type_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,0.730598,0.847703,0.789150
District,76.956733,80.966636,0.704577,0.833901,0.769239
